## Imports

In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
1,Asia\r900 Brickell Key Blvd 3103\rMiami,5/27/2022,8/18/2022,66,"$4,150,000.00",3398,"$1,221.31",Fabiola Lima,KB Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,Asia 900 Brickell Key Blvd 3103 Miami,Asia,900 Brickell Key Blvd 3103 Miami
2,Ocean Three Condo\r18911 Collins Ave 901\rSunn...,2/25/2022,8/17/2022,112,"$3,200,000.00",3945,$811.15,Madeleyne Sutton,One Sotheby's Int'l Realty,Ekaterina Brosda,"Brosda & Bentley Realtors, LLC",Ocean Three Condo 18911 Collins Ave 901 Sunny ...,Ocean Three Condo,18911 Collins Ave 901 Sunny Isles Beach
3,Roney Palace Condo\r2301 Collins Ave 810\rMiam...,6/9/2022,8/16/2022,8,"$2,475,000.00",1480,"$1,672.30",Ileana De La Torre,Douglas Elliman,Kezia Ceccon,Thine Agency,Roney Palace Condo 2301 Collins Ave 810 Miami ...,Roney Palace Condo,2301 Collins Ave 810 Miami Beach
4,Portofino Tower\r300 S Pointe Dr 2606\rMiami B...,7/8/2022,8/15/2022,6,"$2,200,000.00",2040,"$1,078.43",Christian Brandl,London Foster Realty,Piero Ruggeri,COSMORE Florida Corp.,Portofino Tower 300 S Pointe Dr 2606 Miami Beach,Portofino Tower,300 S Pointe Dr 2606 Miami Beach
5,Ocean Two Condo\r19111 Collins Ave 3707\rSunny...,3/5/2022,8/15/2022,82,"$1,562,500.00",2150,$801.28,Gal Kol,Compass Florida LLC,Asi Topaz,"Brokers, LLC",Ocean Two Condo 19111 Collins Ave 3707 Sunny I...,Ocean Two Condo,19111 Collins Ave 3707 Sunny Isles Beach
6,TDR Tower II Condo\r15901 Collins Ave 406\rSun...,5/25/2022,8/17/2022,72,"$1,500,000.00",2354,$637.21,Giulietta Ulloa,BHHS EWM Realty,Bernard Bell,Sunny Realty,TDR Tower II Condo 15901 Collins Ave 406 Sunny...,TDR Tower II Condo,15901 Collins Ave 406 Sunny Isles Beach
7,Key Colony Botanica\r181 Crandon Blvd 205\rKey...,4/20/2022,8/16/2022,47,"$1,480,000.00",1600,$925.00,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",Istvan Zolcsak,One Sotheby's Int'l Realty,Key Colony Botanica 181 Crandon Blvd 205 Key B...,Key Colony Botanica,181 Crandon Blvd 205 Key Biscayne
8,St Tropez Condo\r7330 Ocean Ter Ph-D\rMiami Beach,5/18/2022,8/17/2022,49,"$1,450,000.00",1410,"$1,028.37",Enrique Inurrieta,London Foster Realty,Enrique Inurrieta,London Foster Realty,St Tropez Condo 7330 Ocean Ter Ph-D Miami Beach,St Tropez Condo,7330 Ocean Ter Ph-D Miami Beach
9,Venice\r13647 Deering Bay Dr 121\rCoral Gables,5/6/2022,8/15/2022,55,"$1,350,000.00",2750,$490.91,Hazel Goldman,RE/MAX Advance Realty,Lindsey Lamchick,Douglas Elliman,Venice 13647 Deering Bay Dr 121 Coral Gables,Venice,13647 Deering Bay Dr 121 Coral Gables
10,V0izcayne South Condo\r253 NE 2nd 4408\rMiami,5/27/2022,8/19/2022,30,"$1,250,000.00",1795,$696.38,Maria Pilar Lecha Puig,EXP Realty LLC,Alexandra White,Compass Florida LLC,V0izcayne South Condo 253 NE 2nd 4408 Miami,V,0izcayne South Condo 253 NE 2nd 4408 Miami


In [6]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

## Geocoder Setup and Execution

In [6]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [7]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [8]:
df['loc']

1     (Asia, 900 Brickell Key Blvd APT 3103, Miami, ...
2     (18911 Collins Ave APT 901, Sunny Isles Beach,...
3     (2301 Collins Ave APT 810, Miami Beach, FL 331...
4     (300 South Pointe Dr APT 2606, Miami Beach, FL...
5     (19111 Collins Ave APT 3707, Sunny Isles Beach...
6     (15901 Collins Ave APT 406, Sunny Isles Beach,...
7     (181 Crandon Blvd APT 205, Key Biscayne, FL 33...
8     (7330 Ocean Terrace d, Miami Beach, FL 33141, ...
9     (13647 Deering Bay Dr APT 121, Coral Gables, F...
10    (253 NE 2nd St, Miami, FL 33132, USA, (25.7765...
Name: loc, dtype: object

In [9]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [10]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Asia\r900 Brickell Key Blvd 3103\rMiami,5/27/2022,8/18/2022,66,"$4,150,000.00",3398,"$1,221.31",Fabiola Lima,KB Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,Asia 900 Brickell Key Blvd 3103 Miami,Asia,900 Brickell Key Blvd 3103 Miami,"(Asia, 900 Brickell Key Blvd APT 3103, Miami, ...","(25.7690295, -80.1856979, 0.0)",25.769029,-80.185698,0.0
2,Ocean Three Condo\r18911 Collins Ave 901\rSunn...,2/25/2022,8/17/2022,112,"$3,200,000.00",3945,$811.15,Madeleyne Sutton,One Sotheby's Int'l Realty,Ekaterina Brosda,"Brosda & Bentley Realtors, LLC",Ocean Three Condo 18911 Collins Ave 901 Sunny ...,Ocean Three Condo,18911 Collins Ave 901 Sunny Isles Beach,"(18911 Collins Ave APT 901, Sunny Isles Beach,...","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0
3,Roney Palace Condo\r2301 Collins Ave 810\rMiam...,6/9/2022,8/16/2022,8,"$2,475,000.00",1480,"$1,672.30",Ileana De La Torre,Douglas Elliman,Kezia Ceccon,Thine Agency,Roney Palace Condo 2301 Collins Ave 810 Miami ...,Roney Palace Condo,2301 Collins Ave 810 Miami Beach,"(2301 Collins Ave APT 810, Miami Beach, FL 331...","(25.7987825, -80.1271941, 0.0)",25.798783,-80.127194,0.0
4,Portofino Tower\r300 S Pointe Dr 2606\rMiami B...,7/8/2022,8/15/2022,6,"$2,200,000.00",2040,"$1,078.43",Christian Brandl,London Foster Realty,Piero Ruggeri,COSMORE Florida Corp.,Portofino Tower 300 S Pointe Dr 2606 Miami Beach,Portofino Tower,300 S Pointe Dr 2606 Miami Beach,"(300 South Pointe Dr APT 2606, Miami Beach, FL...","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0
5,Ocean Two Condo\r19111 Collins Ave 3707\rSunny...,3/5/2022,8/15/2022,82,"$1,562,500.00",2150,$801.28,Gal Kol,Compass Florida LLC,Asi Topaz,"Brokers, LLC",Ocean Two Condo 19111 Collins Ave 3707 Sunny I...,Ocean Two Condo,19111 Collins Ave 3707 Sunny Isles Beach,"(19111 Collins Ave APT 3707, Sunny Isles Beach...","(25.9534131, -80.1197571, 0.0)",25.953413,-80.119757,0.0
6,TDR Tower II Condo\r15901 Collins Ave 406\rSun...,5/25/2022,8/17/2022,72,"$1,500,000.00",2354,$637.21,Giulietta Ulloa,BHHS EWM Realty,Bernard Bell,Sunny Realty,TDR Tower II Condo 15901 Collins Ave 406 Sunny...,TDR Tower II Condo,15901 Collins Ave 406 Sunny Isles Beach,"(15901 Collins Ave APT 406, Sunny Isles Beach,...","(25.9240235, -80.1218071, 0.0)",25.924024,-80.121807,0.0
7,Key Colony Botanica\r181 Crandon Blvd 205\rKey...,4/20/2022,8/16/2022,47,"$1,480,000.00",1600,$925.00,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",Istvan Zolcsak,One Sotheby's Int'l Realty,Key Colony Botanica 181 Crandon Blvd 205 Key B...,Key Colony Botanica,181 Crandon Blvd 205 Key Biscayne,"(181 Crandon Blvd APT 205, Key Biscayne, FL 33...","(25.6977458, -80.16075560000002, 0.0)",25.697746,-80.160756,0.0
8,St Tropez Condo\r7330 Ocean Ter Ph-D\rMiami Beach,5/18/2022,8/17/2022,49,"$1,450,000.00",1410,"$1,028.37",Enrique Inurrieta,London Foster Realty,Enrique Inurrieta,London Foster Realty,St Tropez Condo 7330 Ocean Ter Ph-D Miami Beach,St Tropez Condo,7330 Ocean Ter Ph-D Miami Beach,"(7330 Ocean Terrace d, Miami Beach, FL 33141, ...","(25.8589938, -80.1201329, 0.0)",25.858994,-80.120133,0.0
9,Venice\r13647 Deering Bay Dr 121\rCoral Gables,5/6/2022,8/15/2022,55,"$1,350,000.00",2750,$490.91,Hazel Goldman,RE/MAX Advance Realty,Lindsey Lamchick,Douglas Elliman,Venice 13647 Deering Bay Dr 121 Coral Gables,Venice,13647 Deering Bay Dr 121 Coral Gables,"(13647 Deering Bay Dr APT 121, Coral Gables, F...","(25.6348896, -80.2910632, 0.0)",25.634890,-80.291063,0.0
10,V0izcayne South Condo\r253 NE 2nd 4408\rMiami,5/27/2022,8/19/2022,30,"$1,250,000.00",1795,$696.38,Maria Pilar Lecha Puig,EXP Realty LLC,Alexandra White,Compass Florida LLC,V0izcayne South Condo 253 NE 2nd 4408 Miami,V,0izcayne South Condo 253 NE 2nd 4408 Miami,"(253 NE 2nd St, Miami, FL 33132, USA, (25.7765...","(25

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [11]:
df.at[10,'building_name']=('Vizcayne South')
df.at[10,'address_only']=('253 NE 2nd 4408')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('August 14th - August 20th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [17]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesWeekEnding_08_20_2022
